In [1]:
import numpy as np 
import pandas as pd


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('train.csv')[['Age','Fare','SibSp','Parch','Survived']]
df.head(3)


,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1


In [8]:
df.isnull().sum()
# df.fillna(df['Age'].mean(),inplace=True)

Age         0
Fare        0
SibSp       0
Parch       0
Survived    0
dtype: int64

In [9]:
df ['Family'] = df['SibSp'] + df['Parch']

In [11]:
df['Family']
df.drop(columns=['SibSp','Parch'],inplace=True)

In [12]:
df.head(3)

,Age,Fare,Survived,Family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0


In [13]:
x = df.drop(columns=['Survived'])
y = df['Survived']

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
x_train.head(3)

,Age,Fare,Family
331,45.5,28.500,0
733,23.0,13.000,0
382,32.0,7.925,0


In [17]:
## Without binarization 
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
accuracy_score(y_test,y_pred)

0.6927374301675978

In [18]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10, scoring='accuracy'))

np.float64(0.6521098626716604)

In [20]:
## Applying Binarization 
from sklearn.preprocessing import Binarizer
trf = ColumnTransformer([
  ('bin', Binarizer(copy=False),['Family'])
],remainder='passthrough')

In [27]:
trf

d:\machine learning\ml-campus\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('bin', Binarizer(copy=False), ['Family'])])

In [21]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [22]:
pd.DataFrame(x_train_trf, columns=['Family','Age','Fare'])

,Family,Age,Fare
0,0.0,45.500000,28.5000
1,0.0,23.000000,13.0000
2,0.0,32.000000,7.9250
3,1.0,26.000000,7.8542
4,1.0,6.000000,31.2750
...,...,...,...
707,0.0,21.000000,7.6500
708,0.0,29.699118,31.0000
709,1.0,41.000000,14.1083
710,1.0,14.000000,120.0000


In [23]:
clf = DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
y_pred2 = clf.predict(x_test_trf)
accuracy_score(y_test,y_pred2)

0.6871508379888268

In [26]:
x_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),x_trf,y,cv=10 ,scoring='accuracy'))

np.float64(0.6520973782771535)